First, we need a graph. A graph is just a bunch of objects, typically called nodes which are connected to each other. The connections are called edges, and can have a direction, like Tom owes money to Andy, or can be non-directional, like a road connecting two cities.

A graph is a collection of nodes and edges. The BFS algo can tell us if two nodes are connected, and finds the shortest path b/w them.

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import great_circle
from collections import deque

To make the graph interesting, I am using the [airlines and route information database from openflights.org](https://openflights.org/data.html).

So we have two datasets, one about airports, and the routes ones tells us which airports are connected to each other.

In [13]:
cols = ["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude",
        "Timezone", "DST", "Tz database", "Type", "Source"]
airports = pd.read_csv("data/airports.dat", header=None, names=cols)
airports.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


Step one is is to get rid of all the info we don't need for our graph.

So for Airports, I am just keeping:
- **Airport ID**, Unique OpenFlights identifier
- **City**
- **Country**
- **Latitude/Longitude**

In [9]:
# dropping the columns we don't need
keep_cols = ["Airport ID", "City", "Country", "Latitude", "Longitude"]
airports = airports[keep_cols]
airports.head()

,Airport ID,City,Country,Latitude,Longitude
0,1,Goroka,Papua New Guinea,-6.081690,145.391998
1,2,Madang,Papua New Guinea,-5.207080,145.789001
2,3,Mount Hagen,Papua New Guinea,-5.826790,144.296005
3,4,Nadzab,Papua New Guinea,-6.569803,146.725977
4,5,Port Moresby,Papua New Guinea,-9.443380,147.220001


The second data set is the routes flown by airlines.

![](https://openflights.org/demo/openflights-routedb.png)

> As of January 2012, the OpenFlights/Airline Route Mapper Route Database contains 59036 routes between 3209 airports on 531 airlines spanning the globe, as shown in the map above. 

In [38]:
cols = ["Airline", "Airline ID", "Source airport", "Source Airport ID", "Destination airport",
        "Dest Airport ID", "Codeshare", "Stops", "Equipment"]
routes = pd.read_csv("data/routes.dat", header=None, names=cols)
routes.head()

,Airline,Airline ID,Source airport,Source Airport ID,Destination airport,Dest Airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


We just need:

- **Airline ID**: Unique OpenFlights identifier for airline
- **Source Airport ID**: Unique OpenFlights identifier for source airport
- **Dest Airport ID**: Unique OpenFlights identifier for dest airport

In [39]:
# first drop all rows where stops aren't 0, as we only want direct connections
routes = routes[routes["Stops"] == 0]

keep_cols = ["Airline ID", "Source Airport ID", "Dest Airport ID" ]
routes = routes[keep_cols]
routes.head()

,Airline ID,Source Airport ID,Dest Airport ID
0,410,2965,2990
1,410,2966,2990
2,410,2966,2962
3,410,2968,2990
4,410,2968,4078
